In [49]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
from ipywidgets import *

In [51]:
movies = pd.read_csv(r"C:\Users\adars\Desktop\jupyternotebook\movies.csv")
ratings = pd.read_csv(r"C:\Users\adars\Desktop\jupyternotebook\ratings.csv")

In [52]:
movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [53]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [54]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [55]:
movies.shape

(9742, 3)

In [56]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [57]:
ratings.shape

(100836, 4)

In [58]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [59]:
movies['genres']=movies['genres'].str.split("|")

In [60]:
movies2=movies.explode('genres')
movies2

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
...,...,...,...
9738,193583,No Game No Life: Zero (2017),Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Animation


In [61]:
#removal  of no genre columns
movies2.o=movies2[movies2['genres']!='(no genres listed)']
print(movies2['genres'].unique())
print("   ")
print(movies2['genres'].nunique())

['Adventure' 'Animation' 'Children' 'Comedy' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Mystery' 'Sci-Fi' 'War' 'Musical'
 'Documentary' 'IMAX' 'Western' 'Film-Noir' '(no genres listed)']
   
20


C:\Users\adars\AppData\Local\Temp\ipykernel_29100\786911028.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  movies2.o=movies2[movies2['genres']!='(no genres listed)']


In [62]:
ratings = ratings.drop('timestamp', axis=1)


In [63]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [64]:
#grouping based upon genres
#merging movies df and rating
merged_data=pd.merge(ratings,movies2,on=['movieId'],how='inner')
print(merged_data.head())

   userId  movieId  rating             title     genres
0       1        1     4.0  Toy Story (1995)  Adventure
1       1        1     4.0  Toy Story (1995)  Animation
2       1        1     4.0  Toy Story (1995)   Children
3       1        1     4.0  Toy Story (1995)     Comedy
4       1        1     4.0  Toy Story (1995)    Fantasy


In [65]:
#popularity recomender system
############################
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=["Genres","Title","Average Ratings","Number of Ratings"]
popularity

,Genres,Title,Average Ratings,Number of Ratings
0,(no genres listed),A Christmas Story Live! (2017),2.000000,1
1,(no genres listed),A Cosmic Christmas (1977),4.000000,1
2,(no genres listed),A Midsummer Night's Dream (2016),4.000000,1
3,(no genres listed),Ali Wong: Baby Cobra (2016),4.500000,1
4,(no genres listed),Ben-hur (2016),0.500000,1
...,...,...,...,...
22031,Western,Winds of the Wasteland (1936),3.500000,1
22032,Western,Wyatt Earp (1994),3.095238,21
22033,Western,Young Guns (1988),3.100000,25
22034,Western,Young Guns II (1990),3.000000,11


In [66]:
popularity[(popularity['Genres']=='Action')&(popularity['Number of Ratings']>=50)]


,Genres,Title,Average Ratings,Number of Ratings
46,Action,28 Days Later (2002),3.974138,58
54,Action,300 (2007),3.681250,80
71,Action,"Abyss, The (1989)",3.475806,62
89,Action,Air Force One (1997),3.344828,58
100,Action,Aliens (1986),3.964286,126
...,...,...,...,...
1816,Action,Wild Wild West (1999),2.207547,53
1835,Action,X-Men (2000),3.699248,133
1840,Action,X-Men: The Last Stand (2006),3.355769,52
1841,Action,X2: X-Men United (2003),3.723684,76


In [67]:
#sorting by average rating column(sample)
popularity[(popularity['Genres']=='Action')&(popularity['Number of Ratings']>=50)].sort_values(by='Average Ratings',ascending=False).head(7)

,Genres,Title,Average Ratings,Number of Ratings
585,Action,Fight Club (1999),4.272936,218
408,Action,"Dark Knight, The (2008)",4.238255,149
1289,Action,"Princess Bride, The (1987)",4.232394,142
1539,Action,Star Wars: Episode IV - A New Hope (1977),4.231076,251
126,Action,Apocalypse Now (1979),4.219626,107
1540,Action,Star Wars: Episode V - The Empire Strikes Back...,4.215640,211
1313,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.207500,200


In [68]:
#function for movie recommendation based on popularity
######################################################
def TopNPopularMovies(genre,threshold,topN):
    popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=["Genres","Title","Average Ratings","Number of Ratings"]
    #filter the data
    topNrecommendations=popularity[(popularity['Genres']==genre)&(popularity['Number of Ratings']>=threshold)].sort_values(by='Average Ratings',ascending=False).head(topN)
    #output
    topNrecommendations['Sno.']=range(1,len(topNrecommendations)+1)
    topNrecommendations.index=range(0,len(topNrecommendations))
    topNrecommendations.columns=['Genres','Movie Title','Average Movie Rating','Number of Reviews','Sno.']
    return topNrecommendations[['Sno.','Movie Title','Average Movie Rating','Number of Reviews']]

In [69]:
#testing the function(first time)
genre='Adventure'
threshold=50
topN=8
TopNPopularMovies(genre,threshold=threshold,topN=topN)

,Sno.,Movie Title,Average Movie Rating,Number of Reviews
0,1,"Princess Bride, The (1987)",4.232394,142
1,2,Star Wars: Episode IV - A New Hope (1977),4.231076,251
2,3,Star Wars: Episode V - The Empire Strikes Back...,4.215640,211
3,4,Raiders of the Lost Ark (Indiana Jones and the...,4.207500,200
4,5,North by Northwest (1959),4.184211,57
5,6,Monty Python and the Holy Grail (1975),4.161765,136
6,7,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.155172,87
7,8,City of God (Cidade de Deus) (2002),4.146667,75


In [70]:
#content based recommendation system
#############################################


In [71]:
movies2.groupby('title').agg({"genres":lambda x:" ".join(list(x))})


,genres
title,
'71 (2014),Action Drama Thriller War
'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
'Round Midnight (1986),Drama Musical
'Salem's Lot (2004),Drama Horror Mystery Thriller
'Til There Was You (1997),Drama Romance
...,...
eXistenZ (1999),Action Sci-Fi Thriller
xXx (2002),Action Crime Thriller
xXx: State of the Union (2005),Action Crime Thriller


In [72]:
movies3=movies2.groupby('title').agg({"genres":lambda x:" ".join(list(x))}).reset_index()
movies3.head()


,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Salem's Lot (2004),Drama Horror Mystery Thriller
4,'Til There Was You (1997),Drama Romance


In [73]:
#instance of tfidvectorizer 
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf.fit_transform(movies3['genres'])


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 43100 stored elements and shape (9737, 599)>

In [74]:
tf_matrix=tf.fit_transform(movies3['genres'])


In [75]:
#cosine similiarity
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)
cosine_sim


array([[1.        , 0.02473499, 0.03169578, ..., 0.09695131, 0.        ,
        0.        ],
       [0.02473499, 1.        , 0.        , ..., 0.03417082, 0.02127466,
        0.02566391],
       [0.03169578, 0.        , 1.        , ..., 0.        , 0.        ,
        0.36167587],
       ...,
       [0.09695131, 0.03417082, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02127466, 0.        , ..., 0.        , 1.        ,
        0.05944205],
       [0.        , 0.02566391, 0.36167587, ..., 0.        , 0.05944205,
        1.        ]])

In [76]:
#indices=pd.Series(movies3.index,index=movies3['title'])
#indices

In [77]:
#index=indices["'71 (2014)"]
#index

In [78]:
#scores=list(enumerate(cosine_sim[index]))
#scores

In [79]:
#matched=sorted(scores,key=lambda x:x[1],reverse=True)[:5]

In [80]:
#matched = [i[0] for i in matched]
#matched

In [81]:
#movies3.iloc[matched]

In [82]:
#final function 
def recommendation_genre(movie_df,similarity_matrix,movie_title,topN):
    #indices for movies
    ###############################################
    indices=pd.Series(movies3.index,index=movies3['title'])
    ####################################################
    #index of target movie
    index=indices[movie_title]
    cosine_scores=list(enumerate(similarity_matrix[index]))
    cosine_scores=sorted(cosine_scores,key=lambda x:x[1],reverse=True)[1:topN+2]
    #extract matching movies
    matched = [i[0] for i in cosine_scores]
    matching_df=movie_df.iloc[matched]
    #filter out targert movie
    matching_df=matching_df[matching_df['title']!=movie_title]
    #output
    matching_df.rename(columns={'title':'Movie Title'},inplace=True)
    matching_df['Sno.']=range(1,len(matching_df)+1)
    matching_df.index=range(0,len(matching_df))
    return matching_df[['Sno.','Movie Title']].head(topN)
    




In [83]:
#test case for func:recommendation_genre
recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title='Toy Story (1995)',topN=10)

,Sno.,Movie Title
0,1,Antz (1998)
1,2,Asterix and the Vikings (Astérix et les Viking...
2,3,"Emperor's New Groove, The (2000)"
3,4,Moana (2016)
4,5,"Monsters, Inc. (2001)"
5,6,Shrek the Third (2007)
6,7,"Tale of Despereaux, The (2008)"
7,8,The Good Dinosaur (2015)
8,9,Toy Story 2 (1999)
9,10,Turbo (2013)


In [84]:
#interactive widgets
#popularity based
genres=Dropdown(options=list(set(movies2['genres'])),description="Genres",style={"description_width":'initial'})
num_reviews=IntText(description="Minimum Reviews",style={"description_width":'initial'})
num_recommendations_1=IntText(description="Number of Recommendations",style={"description_width":'initial'})
#tabs
b1=Button(description="RECOMMEND ME",style={"description_width":'initial'})
h1=HBox([num_reviews,num_recommendations_1])
popularity_tab=VBox([genres,h1,b1])

#interactive widgets
#content based
title=Textarea(description="Movie Title",style={"description_width":'initial'})
num_recommendations_2=IntText(description="Number of Recommendations",style={"description_width":'initial'})
#tabs
h2=HBox([title,num_recommendations_2])
b2=Button(description="RECOMMEND ME",style={"description_width":'initial'})
# Changed from list to VBox to make it a proper widget
content_tab=VBox([h2,b2])  # Changed from list to VBox


#creating final tabs
tabs=[popularity_tab,content_tab]
wid=widgets.Tab(tabs)
#set titles to the tab
names=['Popularity Based Recommendations','Content Based Recommendations']
[wid.set_title(i,title) for i,title in enumerate(names)]
display(wid)

In [85]:
#setting up events to respond when clicked upon
#popularity
def b1_clicked(b):
    global output
    output=TopNPopularMovies(genre=genres.value,threshold=num_reviews.value,topN=num_recommendations_1.value)
b1.on_click(b1_clicked)

#content
def b2_clicked(b):
    global output
    result=recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title=title.value,topN=10)
    output=result
b2.on_click(b2_clicked)




    

In [98]:
display(wid)

In [94]:
output


,Sno.,Movie Title
0,1,Aladdin and the King of Thieves (1996)
1,2,Frosty the Snowman (1969)
2,3,Tangled (2010)
3,4,"Adventures of Rocky and Bullwinkle, The (2000)"
4,5,Antz (1998)
5,6,Asterix and the Vikings (Astérix et les Viking...
6,7,"Emperor's New Groove, The (2000)"
7,8,Moana (2016)
8,9,"Monsters, Inc. (2001)"
9,10,Shrek the Third (2007)
